In [1]:
# result: address, holdings, previousTradeTimestamp, boughtTotal, holdingCurrently

import pandas as pd
import alchemyHelper
import graphqlHelper
import json

# GLOBAL VARIABLES
uniswapV2Router = "0x7a250d5630b4cf539739df2c5dacb4c659f2488d"
uniswapV3Router = "0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45"
weiDivisible = 1000000000000000000

# getting count
with open("mainCOUNT.txt", "r") as f:
    rowNum = int(f.read())
count = rowNum + 1

walletsList = pd.read_csv("wallets_test.csv")
walletsList = walletsList.drop_duplicates()

tokensTraded = 0
stillHoldingOrDusted = 0
sellingIncrementally = 0
accountTransferORNULL = 0
stablecoinTransfer = 0
buysTOTAL = []
sellsTOTAL = []
profits = {}

for i, row in walletsList[rowNum:].iterrows():
    bigDICT = {}
    wallet = row["Address"]
    print(f"CURRENT WALLET: {wallet}")

    # BUY SECTION
    buyList = alchemyHelper.getBuys(wallet)
    for dict in buyList:
        txnHash = dict["hash"]
        contract = dict["rawContract"]["address"]

        if contract not in bigDICT:
            bigDICT[contract] = {
                "buys": [],
                "sells": []
            }

        bigDICT[contract]['buys'].append(txnHash)

    # SELL SECTION
    sellList = alchemyHelper.getSells(wallet)
    for dict in sellList:
        txnHash = dict["hash"]
        contract = dict["rawContract"]["address"]

        if contract not in bigDICT:
            bigDICT[contract] = {
                "buys": [],
                "sells": []
            }

        bigDICT[contract]['sells'].append(txnHash)

    # bigDICT[contract]
    # ITERATING BIG DICTIONARY
    for contract in bigDICT:
        # REMOVING DUPLICATES
        bigDICT[contract]["buys"] = [*set(bigDICT[contract]["buys"])]
        bigDICT[contract]["sells"] = [*set(bigDICT[contract]["sells"])]

        # CASE0 - IRREGULARITY
        if (bigDICT[contract]["buys"] == []):
            continue

        # CASE1 - still holding token or rugged/dusted
        if (bigDICT[contract]["sells"] == []):
            stillHoldingOrDusted += 1
            continue

        # CASE2 - Great buy and selling incrementally, smart trader
        buyLen = len(bigDICT[contract]["buys"])
        sellLen = len(bigDICT[contract]["sells"])
        diffLen = sellLen / float(buyLen)
        if (diffLen > 2):
            sellingIncrementally += 1

        excludeThisContract = False
        buys = 0.0
        # CHECKING BUYS GRAPHQL
        for buy in bigDICT[contract]["buys"]:
            print("TXN: ", buy)
            result = graphqlHelper.getData(buy)
            if (result == -1):
                accountTransferORNULL += 1
                excludeThisContract = True
            elif (result == -2):
                stablecoinTransfer += 1
            else:
                buys += result

        sells = 0.0
        # CHECKING SELLS GRAPHQL
        for sell in bigDICT[contract]["sells"]:
            result = graphqlHelper.getData(sell)
            if (result == -1):
                accountTransferORNULL += 1
                excludeThisContract = True
            elif (result == -2):
                stablecoinTransfer += 1
            else:
                sells += result

        profit = sells - buys
        # STABLECOIN OR JUST TRANSFER
        if (profit == 0.0 or excludeThisContract):
            continue

        buysTOTAL.append(buys)
        sellsTOTAL.append(sells)
        profits[contract] = profit
        tokensTraded += 1

    #with open("testANDY2.json", "w") as file:
    #    json.dump(profits, file)

    profitsAVG = sum(profits.values())/len(profits)

    df = pd.DataFrame(columns=list(['Wallet_Address','Tokens_Traded', 'Still_Holding_OR_Dusted', 'Sold_Incrementally_(NotJet)', 'Transfer_To_Other_Account_OR_NULL', 'Transfer_Stablecoin', 'Average_Profit']))
    df.loc[count] = [wallet,tokensTraded, stillHoldingOrDusted, sellingIncrementally, accountTransferORNULL, stablecoinTransfer, profitsAVG]

    count += 1

df

   


CURRENT WALLET: 0x4167691c4c862084039ac5e58f70fe859465163b
TXN:  0x9f92dacd23766348a04c53e44d36a9bd099535c3a95e67764e98870f59b3b6fc
TXN:  0x6115acd173e5181d756781848806ddd727daed971495a67faeef252187770eaa
TXN:  0x7e3d3cc871fc3eba892fc1debcdfc5c7f614629036d5c87bfac0220ddae5b505
TXN:  0xef302e20097be6a0c51255cb01fcdb47b5e3a91d7aa3e6daa090b7cc27cca209
TXN:  0xb7b7ac5dee65025672b1b5796fec2fb302da0d39a2703d976dbf207ea630c9c6
TXN:  0xe9b1f6d06bd502d84f05f3db4c9258db39c68be8eab645c3ae91d6e497e77f9e
TXN:  0x4f8364602739cf678571e60a1efe44d355a3cce785afe84f2808ba57dff0b39f
TXN:  0xe5b2f907640c9c8b5bce335f5b97cf4e88f310a709ba3ae5c0fff8726840228b
TXN:  0x036d5c9787bf3b0c8bd10087adbf50782386c7cb26beb8f59563b50776729a2c
TXN:  0x6afefdcc17caab29c174f0df0caaf386346be58dc14932dd73f30d54faf47601
TXN:  0xf904aa13e8383445f64f976ec6648b34220dd5cdc22b6d7375c391b8521b05f0
TXN:  0x8305acacbd0fd1241627cd3d9cfc654b16e4f7f3bfe681a0e78d701a4e9af947
TXN:  0x2cadfa5d1f29bb00acd3bfcace94d9ef781b851c8e63b02afa7b3838a

,Wallet_Address,Tokens_Traded,Still_Holding_OR_Dusted,Sold_Incrementally_(NotJet),Transfer_To_Other_Account_OR_NULL,Transfer_Stablecoin,Average_Profit
1,0x4167691c4c862084039ac5e58f70fe859465163b,8,47,3,38,0,1818.542417
